                                                  # Practical -03
B-76 
SHIVAM GUPTA 


Problem Statement :Write a program to implement Breadth First Search andA* to solve 8-Puzzle problem

BFS: 

In [1]:
import sys
import numpy as np


class Node:
	def __init__(self, state, parent, action):
		self.state = state
		self.parent = parent
		self.action = action


class StackFrontier:
	def __init__(self):
		self.frontier = []

	def add(self, node):
		self.frontier.append(node)

	def contains_state(self, state):
		return any((node.state[0] == state[0]).all() for node in self.frontier)
	
	def empty(self):
		return len(self.frontier) == 0
	
	def remove(self):
		if self.empty():
			raise Exception("Empty Frontier")
		else:
			node = self.frontier[-1]
			self.frontier = self.frontier[:-1]
			return node


class QueueFrontier(StackFrontier):
	def remove(self):
		if self.empty():
			raise Exception("Empty Frontier")
		else:
			node = self.frontier[0]
			self.frontier = self.frontier[1:]
			return node


class Puzzle:
	def __init__(self, start, startIndex, goal, goalIndex):
		self.start = [start, startIndex]
		self.goal = [goal, goalIndex] 
		self.solution = None

	def neighbors(self, state):
		mat, (row, col) = state
		results = []
		
		if row > 0:
			mat1 = np.copy(mat)
			mat1[row][col] = mat1[row - 1][col]
			mat1[row - 1][col] = 0
			results.append(('up', [mat1, (row - 1, col)]))
		if col > 0:
			mat1 = np.copy(mat)
			mat1[row][col] = mat1[row][col - 1]
			mat1[row][col - 1] = 0
			results.append(('left', [mat1, (row, col - 1)]))
		if row < 2:
			mat1 = np.copy(mat)
			mat1[row][col] = mat1[row + 1][col]
			mat1[row + 1][col] = 0
			results.append(('down', [mat1, (row + 1, col)]))
		if col < 2:
			mat1 = np.copy(mat)
			mat1[row][col] = mat1[row][col + 1]
			mat1[row][col + 1] = 0
			results.append(('right', [mat1, (row, col + 1)]))

		return results

	def print(self):
		solution = self.solution if self.solution is not None else None
		print("Start State:\n", self.start[0], "\n")
		print("Goal State:\n",  self.goal[0], "\n")
		print("\nStates Explored: ", self.num_explored, "\n")
		print("Solution:\n ")
		for action, cell in zip(solution[0], solution[1]):
			print("action: ", action, "\n", cell[0], "\n")
		print("Goal Reached!!")

	def does_not_contain_state(self, state):
		for st in self.explored:
			if (st[0] == state[0]).all():
				return False
		return True
	
	def solve(self):
		self.num_explored = 0

		start = Node(state=self.start, parent=None, action=None)
		frontier = QueueFrontier()
		frontier.add(start)

		self.explored = [] 

		while True:
			if frontier.empty():
				raise Exception("No solution")

			node = frontier.remove()
			self.num_explored += 1

			if (node.state[0] == self.goal[0]).all():
				actions = []
				cells = []
				while node.parent is not None:
					actions.append(node.action)
					cells.append(node.state)
					node = node.parent
				actions.reverse()
				cells.reverse()
				self.solution = (actions,  cells)
				return

			self.explored.append(node.state)

			for action, state in self.neighbors(node.state):
				if not frontier.contains_state(state) and self.does_not_contain_state(state):
					child = Node(state=state, parent=node, action=action)
					frontier.add(child)


start = np.array([[1, 2, 3], [8, 0, 4], [7, 6, 5]])
goal = np.array([[2, 8, 1], [0, 4, 3], [7, 6, 5]])


startIndex = (1, 1)
goalIndex = (1, 0)


p = Puzzle(start, startIndex, goal, goalIndex)
p.solve()
p.print()

Start State:
 [[1 2 3]
 [8 0 4]
 [7 6 5]] 

Goal State:
 [[2 8 1]
 [0 4 3]
 [7 6 5]] 


States Explored:  358 

Solution:
 
action:  up 
 [[1 0 3]
 [8 2 4]
 [7 6 5]] 

action:  left 
 [[0 1 3]
 [8 2 4]
 [7 6 5]] 

action:  down 
 [[8 1 3]
 [0 2 4]
 [7 6 5]] 

action:  right 
 [[8 1 3]
 [2 0 4]
 [7 6 5]] 

action:  right 
 [[8 1 3]
 [2 4 0]
 [7 6 5]] 

action:  up 
 [[8 1 0]
 [2 4 3]
 [7 6 5]] 

action:  left 
 [[8 0 1]
 [2 4 3]
 [7 6 5]] 

action:  left 
 [[0 8 1]
 [2 4 3]
 [7 6 5]] 

action:  down 
 [[2 8 1]
 [0 4 3]
 [7 6 5]] 

Goal Reached!!


A* SEARCH

In [12]:
from queue import PriorityQueue
from time import time
start_state, endState = [7, 2, 4, 5, 0, 6,
                            8, 3, 1], [9, 1, 2, 3, 4, 5, 6, 7, 8]
class Node:
    goal_state = [0, 1, 2, 3, 4, 5, 6, 7, 8]
    heuristic = None
    eval_func = None
    require_heuristic = False
    nodes_generated = 0
    def __init__(self, state, parent, action, path_cost,require_heuristic=False):
        self.parent = parent
        self.state = state
        self.action = action
        if parent:
            self.path_cost = parent.path_cost + path_cost
        else:
             self.path_cost = path_cost
        if require_heuristic:
            self.require_heuristic = True
            self.generate_heuristic()
            self.eval_func = self.heuristic+self.path_cost
        Node.nodes_generated += 1
    def __str__(self):
        return str(self.state[0:3])+'\n'+str(self.state[3:6])+'\n'+str(self.state[6:9])
    def find_answer(self):
        solution = []
        solution.append(self.action)
        path = self
        while path.parent != None:
            path = path.parent
            solution.append(path.action)
        solution = solution[:-1]
        solution.reverse()
        return solution
    def produce_child(self):
        children = []
        x = self.state.index(0)
        i = int(x / 3)
        j = int(x % 3)
        isPossibeMoves = self.find_isPossibeMoves(i, j)
        for action in isPossibeMoves:
            temp = self.state.copy()
            if action == 'U':
                temp[x], temp[x-3] = temp[x-3], temp[x]
            elif action == 'D':
                temp[x], temp[x+3] = temp[x+3], temp[x]
            elif action == 'L':
                temp[x], temp[x-1] = temp[x-1], temp[x]
            elif action == 'R':
                temp[x], temp[x+1] = temp[x+1], temp[x]
            children.append(Node(temp, self, action, 1, self.require_heuristic))
        return children
    def generate_heuristic(self):
        self.heuristic = 0
        for num in range(1, 9):
            distance = abs(self.state.index(num) -
            self.goal_state.index(num))
            i = int(distance/3)
            j = int(distance % 3)
            self.heuristic = self.heuristic+i+j
    def goal_test(self):
        if self.state == self.goal_state:
            return True
        return False
    @staticmethod
    def find_isPossibeMoves(i, j):
        isPossibeMove = ['U', 'D', 'L', 'R']
        if i == 0:
            isPossibeMove.remove('U')
        elif i == 2:
            isPossibeMove.remove('D')
        if j == 0:
            isPossibeMove.remove('L')
        elif j == 2:
            isPossibeMove.remove('R')
        return isPossibeMove
def DoSearch(initial_state):
    count = 0
    explored = []
    start_node = Node(initial_state, None, None, 0, True)
    q = PriorityQueue()
    q.put((start_node.eval_func, count, start_node))
    while not q.empty():
        node = q.get()
        node = node[2]
        explored.append(node.state)
        if node.goal_test():
            return node.find_answer()
        children = node.produce_child()
        for child in children:
            print("State -> ", child.state," Heuristic value -> ", child.heuristic)
            if child.state not in explored:
                count += 1
                q.put((child.eval_func, count, child))
    return



In [13]:
Node.nodes_generated = 0
t0 = time()
result = DoSearch(start_state)
t1 = time() - t0
print("\nGoal state reached !!")
print("Number of nodes generated : ", Node.nodes_generated - 1)
print("Total time taken : ", t1, end=" sec\n")


State ->  [7, 0, 4, 5, 2, 6, 8, 3, 1]  Heuristic value ->  17
State ->  [7, 2, 4, 5, 3, 6, 8, 0, 1]  Heuristic value ->  15
State ->  [7, 2, 4, 0, 5, 6, 8, 3, 1]  Heuristic value ->  15
State ->  [7, 2, 4, 5, 6, 0, 8, 3, 1]  Heuristic value ->  17
State ->  [7, 2, 4, 5, 0, 6, 8, 3, 1]  Heuristic value ->  16
State ->  [7, 2, 4, 5, 3, 6, 0, 8, 1]  Heuristic value ->  14
State ->  [7, 2, 4, 5, 3, 6, 8, 1, 0]  Heuristic value ->  14
State ->  [0, 2, 4, 7, 5, 6, 8, 3, 1]  Heuristic value ->  14
State ->  [7, 2, 4, 8, 5, 6, 0, 3, 1]  Heuristic value ->  16
State ->  [7, 2, 4, 5, 0, 6, 8, 3, 1]  Heuristic value ->  16
State ->  [7, 2, 4, 0, 3, 6, 5, 8, 1]  Heuristic value ->  13
State ->  [7, 2, 4, 5, 3, 6, 8, 0, 1]  Heuristic value ->  15
State ->  [7, 2, 4, 5, 3, 0, 8, 1, 6]  Heuristic value ->  15
State ->  [7, 2, 4, 5, 3, 6, 8, 0, 1]  Heuristic value ->  15
State ->  [7, 2, 4, 0, 5, 6, 8, 3, 1]  Heuristic value ->  15
State ->  [2, 0, 4, 7, 5, 6, 8, 3, 1]  Heuristic value ->  15
State ->

State ->  [7, 0, 2, 5, 1, 4, 8, 6, 3]  Heuristic value ->  13
State ->  [7, 2, 4, 0, 6, 1, 5, 8, 3]  Heuristic value ->  15
State ->  [2, 0, 4, 7, 6, 1, 5, 8, 3]  Heuristic value ->  15
State ->  [7, 0, 4, 6, 2, 1, 5, 8, 3]  Heuristic value ->  15
State ->  [7, 2, 4, 6, 8, 1, 5, 0, 3]  Heuristic value ->  15
State ->  [7, 2, 4, 0, 6, 1, 5, 8, 3]  Heuristic value ->  15
State ->  [7, 2, 4, 6, 1, 0, 5, 8, 3]  Heuristic value ->  13
State ->  [7, 2, 6, 0, 4, 5, 8, 3, 1]  Heuristic value ->  13
State ->  [2, 0, 6, 7, 4, 5, 8, 3, 1]  Heuristic value ->  13
State ->  [2, 4, 6, 0, 3, 5, 7, 8, 1]  Heuristic value ->  11
State ->  [2, 4, 6, 7, 3, 5, 8, 0, 1]  Heuristic value ->  13
State ->  [2, 4, 6, 7, 3, 0, 8, 1, 5]  Heuristic value ->  13
State ->  [2, 4, 6, 7, 3, 5, 8, 0, 1]  Heuristic value ->  13
State ->  [2, 4, 6, 0, 7, 5, 8, 3, 1]  Heuristic value ->  13
State ->  [4, 0, 6, 2, 7, 5, 8, 3, 1]  Heuristic value ->  13
State ->  [3, 4, 6, 2, 7, 0, 5, 8, 1]  Heuristic value ->  11
State ->

State ->  [7, 8, 2, 3, 4, 0, 5, 1, 6]  Heuristic value ->  11
State ->  [7, 2, 4, 0, 1, 8, 3, 5, 6]  Heuristic value ->  13
State ->  [7, 2, 4, 3, 1, 8, 5, 0, 6]  Heuristic value ->  11
State ->  [7, 6, 0, 5, 3, 2, 8, 1, 4]  Heuristic value ->  16
State ->  [7, 6, 2, 5, 3, 4, 8, 1, 0]  Heuristic value ->  14
State ->  [7, 6, 2, 5, 0, 3, 8, 1, 4]  Heuristic value ->  16
State ->  [7, 6, 2, 5, 0, 1, 8, 4, 3]  Heuristic value ->  16
State ->  [7, 6, 2, 5, 4, 1, 0, 8, 3]  Heuristic value ->  14
State ->  [7, 6, 2, 5, 4, 1, 8, 3, 0]  Heuristic value ->  14
State ->  [7, 0, 2, 8, 5, 4, 3, 1, 6]  Heuristic value ->  13
State ->  [7, 5, 2, 8, 1, 4, 3, 0, 6]  Heuristic value ->  13
State ->  [7, 5, 2, 0, 8, 4, 3, 1, 6]  Heuristic value ->  13
State ->  [7, 5, 2, 8, 4, 0, 3, 1, 6]  Heuristic value ->  13
State ->  [7, 2, 4, 0, 1, 5, 8, 3, 6]  Heuristic value ->  13
State ->  [7, 2, 4, 8, 1, 5, 3, 0, 6]  Heuristic value ->  13
State ->  [7, 3, 0, 5, 1, 2, 8, 6, 4]  Heuristic value ->  14
State ->

State ->  [3, 5, 0, 7, 4, 2, 8, 1, 6]  Heuristic value ->  12
State ->  [3, 5, 2, 7, 4, 6, 8, 1, 0]  Heuristic value ->  10
State ->  [3, 5, 2, 7, 0, 4, 8, 1, 6]  Heuristic value ->  12
State ->  [7, 6, 2, 3, 0, 1, 5, 4, 8]  Heuristic value ->  10
State ->  [0, 7, 2, 3, 6, 1, 5, 4, 8]  Heuristic value ->  8
State ->  [7, 2, 0, 3, 6, 1, 5, 4, 8]  Heuristic value ->  10
State ->  [7, 6, 0, 3, 1, 2, 5, 4, 8]  Heuristic value ->  10
State ->  [7, 6, 2, 3, 1, 8, 5, 4, 0]  Heuristic value ->  10
State ->  [7, 6, 2, 3, 0, 1, 5, 4, 8]  Heuristic value ->  10
State ->  [5, 7, 2, 0, 4, 1, 6, 8, 3]  Heuristic value ->  11
State ->  [5, 7, 2, 6, 4, 1, 8, 0, 3]  Heuristic value ->  13
State ->  [5, 7, 2, 0, 1, 4, 3, 8, 6]  Heuristic value ->  11
State ->  [7, 0, 2, 5, 1, 4, 3, 8, 6]  Heuristic value ->  11
State ->  [5, 7, 2, 0, 4, 6, 3, 8, 1]  Heuristic value ->  11
State ->  [5, 7, 2, 3, 4, 6, 8, 0, 1]  Heuristic value ->  11
State ->  [7, 4, 1, 0, 3, 2, 5, 6, 8]  Heuristic value ->  9
State ->  

State ->  [5, 0, 4, 2, 8, 6, 3, 7, 1]  Heuristic value ->  13
State ->  [7, 4, 6, 2, 0, 1, 5, 8, 3]  Heuristic value ->  14
State ->  [0, 7, 6, 2, 4, 1, 5, 8, 3]  Heuristic value ->  12
State ->  [7, 6, 0, 2, 4, 1, 5, 8, 3]  Heuristic value ->  14
State ->  [7, 4, 0, 2, 1, 6, 5, 8, 3]  Heuristic value ->  12
State ->  [7, 4, 6, 2, 1, 3, 5, 8, 0]  Heuristic value ->  12
State ->  [7, 4, 6, 2, 0, 1, 5, 8, 3]  Heuristic value ->  14
State ->  [7, 4, 1, 5, 0, 2, 8, 6, 3]  Heuristic value ->  14
State ->  [0, 7, 1, 5, 4, 2, 8, 6, 3]  Heuristic value ->  12
State ->  [7, 1, 0, 5, 4, 2, 8, 6, 3]  Heuristic value ->  12
State ->  [0, 4, 1, 7, 5, 2, 8, 6, 3]  Heuristic value ->  12
State ->  [7, 4, 1, 8, 5, 2, 0, 6, 3]  Heuristic value ->  14
State ->  [7, 4, 1, 5, 0, 2, 8, 6, 3]  Heuristic value ->  14
State ->  [3, 4, 0, 2, 7, 1, 5, 6, 8]  Heuristic value ->  8
State ->  [3, 4, 1, 2, 7, 8, 5, 6, 0]  Heuristic value ->  8
State ->  [3, 4, 1, 2, 0, 7, 5, 6, 8]  Heuristic value ->  8
State ->  [

...

State ->  [7, 1, 5, 3, 8, 2, 6, 0, 4]  Heuristic value ->  9
State ->  [7, 0, 2, 3, 5, 1, 6, 8, 4]  Heuristic value ->  9
State ->  [7, 5, 2, 3, 8, 1, 6, 0, 4]  Heuristic value ->  11
State ->  [7, 5, 2, 0, 3, 1, 6, 8, 4]  Heuristic value ->  11
State ->  [7, 5, 2, 3, 1, 0, 6, 8, 4]  Heuristic value ->  9
State ->  [7, 5, 2, 3, 8, 0, 6, 4, 1]  Heuristic value ->  11
State ->  [7, 5, 2, 3, 8, 1, 6, 0, 4]  Heuristic value ->  11
State ->  [7, 3, 2, 4, 0, 1, 5, 6, 8]  Heuristic value ->  10
State ->  [0, 7, 2, 4, 3, 1, 5, 6, 8]  Heuristic value ->  8
State ->  [7, 2, 0, 4, 3, 1, 5, 6, 8]  Heuristic value ->  10
State ->  [7, 3, 0, 4, 1, 2, 5, 6, 8]  Heuristic value ->  10
State ->  [7, 3, 2, 4, 1, 8, 5, 6, 0]  Heuristic value ->  10
State ->  [7, 3, 2, 4, 0, 1, 5, 6, 8]  Heuristic value ->  10
State ->  [6, 5, 2, 0, 1, 3, 7, 8, 4]  Heuristic value ->  11
State ->  [5, 0, 2, 6, 1, 3, 7, 8, 4]  Heuristic value ->  11
State ->  [3, 1, 5, 0, 7, 2, 8, 6, 4]  Heuristic value ->  9
State ->  [1,

State ->  [4, 7, 2, 6, 1, 3, 0, 5, 8]  Heuristic value ->  10
State ->  [4, 7, 2, 6, 1, 3, 5, 8, 0]  Heuristic value ->  10
State ->  [2, 1, 4, 0, 8, 5, 3, 7, 6]  Heuristic value ->  9
State ->  [1, 0, 4, 2, 8, 5, 3, 7, 6]  Heuristic value ->  9
State ->  [6, 5, 2, 0, 1, 3, 4, 7, 8]  Heuristic value ->  9
State ->  [6, 5, 2, 4, 1, 3, 7, 0, 8]  Heuristic value ->  9
State ->  [2, 1, 4, 6, 3, 0, 5, 7, 8]  Heuristic value ->  7
State ->  [2, 0, 1, 6, 3, 4, 5, 7, 8]  Heuristic value ->  7
State ->  [2, 1, 3, 5, 4, 0, 7, 8, 6]  Heuristic value ->  9
State ->  [2, 1, 3, 5, 4, 6, 7, 0, 8]  Heuristic value ->  7
State ->  [2, 4, 1, 0, 3, 6, 5, 7, 8]  Heuristic value ->  7
State ->  [2, 4, 1, 5, 3, 6, 7, 0, 8]  Heuristic value ->  9
State ->  [4, 1, 3, 7, 0, 2, 6, 5, 8]  Heuristic value ->  8
State ->  [4, 1, 3, 7, 5, 2, 0, 6, 8]  Heuristic value ->  8
State ->  [4, 1, 3, 7, 5, 2, 6, 8, 0]  Heuristic value ->  8
State ->  [0, 1, 3, 4, 7, 2, 6, 5, 8]  Heuristic value ->  6
State ->  [4, 1, 3, 6,

State ->  [7, 0, 2, 5, 3, 4, 1, 6, 8]  Heuristic value ->  11
State ->  [0, 3, 2, 7, 4, 6, 8, 1, 5]  Heuristic value ->  10
State ->  [7, 3, 2, 8, 4, 6, 0, 1, 5]  Heuristic value ->  12
State ->  [7, 3, 2, 4, 0, 6, 8, 1, 5]  Heuristic value ->  12
State ->  [7, 5, 2, 1, 0, 4, 3, 6, 8]  Heuristic value ->  10
State ->  [0, 7, 2, 1, 5, 4, 3, 6, 8]  Heuristic value ->  8
State ->  [7, 2, 0, 1, 5, 4, 3, 6, 8]  Heuristic value ->  10
State ->  [6, 7, 2, 0, 3, 1, 5, 8, 4]  Heuristic value ->  11
State ->  [7, 0, 2, 6, 3, 1, 5, 8, 4]  Heuristic value ->  11
State ->  [5, 3, 2, 6, 0, 4, 8, 1, 7]  Heuristic value ->  12
State ->  [0, 5, 2, 6, 3, 4, 8, 1, 7]  Heuristic value ->  10
State ->  [5, 2, 0, 6, 3, 4, 8, 1, 7]  Heuristic value ->  12
State ->  [5, 1, 2, 3, 0, 4, 8, 6, 7]  Heuristic value ->  8
State ->  [0, 5, 2, 3, 1, 4, 8, 6, 7]  Heuristic value ->  8
State ->  [5, 2, 0, 3, 1, 4, 8, 6, 7]  Heuristic value ->  10
State ->  [4, 2, 5, 0, 3, 6, 7, 8, 1]  Heuristic value ->  11
State ->  [

State ->  [4, 1, 2, 3, 6, 0, 5, 7, 8]  Heuristic value ->  5
State ->  [3, 4, 2, 0, 1, 6, 5, 7, 8]  Heuristic value ->  5
State ->  [4, 0, 2, 3, 1, 6, 5, 7, 8]  Heuristic value ->  5
State ->  [2, 5, 1, 0, 4, 6, 3, 7, 8]  Heuristic value ->  7
State ->  [5, 0, 1, 2, 4, 6, 3, 7, 8]  Heuristic value ->  7
State ->  [5, 1, 6, 2, 4, 0, 3, 7, 8]  Heuristic value ->  7
State ->  [5, 0, 1, 2, 4, 6, 3, 7, 8]  Heuristic value ->  7
State ->  [4, 0, 2, 3, 1, 6, 5, 7, 8]  Heuristic value ->  5
State ->  [4, 1, 2, 3, 7, 6, 5, 0, 8]  Heuristic value ->  5
State ->  [4, 1, 2, 0, 3, 6, 5, 7, 8]  Heuristic value ->  5
State ->  [4, 1, 2, 3, 6, 0, 5, 7, 8]  Heuristic value ->  5
State ->  [3, 4, 2, 0, 1, 6, 5, 7, 8]  Heuristic value ->  5
State ->  [4, 0, 2, 3, 1, 6, 5, 7, 8]  Heuristic value ->  5
State ->  [3, 1, 2, 7, 0, 4, 6, 5, 8]  Heuristic value ->  6
State ->  [3, 1, 2, 7, 5, 4, 0, 6, 8]  Heuristic value ->  6
State ->  [3, 1, 2, 7, 5, 4, 6, 8, 0]  Heuristic value ->  6
State ->  [3, 1, 0, 7, 4

State ->  [3, 1, 2, 0, 8, 4, 6, 7, 5]  Heuristic value ->  5
State ->  [1, 0, 2, 3, 8, 4, 6, 7, 5]  Heuristic value ->  5
State ->  [1, 8, 2, 3, 4, 0, 6, 7, 5]  Heuristic value ->  5
State ->  [1, 8, 2, 3, 4, 5, 6, 0, 7]  Heuristic value ->  5
State ->  [3, 1, 2, 0, 5, 4, 7, 6, 8]  Heuristic value ->  5
State ->  [1, 0, 2, 3, 5, 4, 7, 6, 8]  Heuristic value ->  5
State ->  [7, 6, 1, 0, 4, 3, 5, 8, 2]  Heuristic value ->  13
State ->  [6, 0, 1, 7, 4, 3, 5, 8, 2]  Heuristic value ->  11
State ->  [2, 6, 1, 0, 4, 3, 7, 8, 5]  Heuristic value ->  11
State ->  [6, 0, 1, 2, 4, 3, 7, 8, 5]  Heuristic value ->  9
State ->  [7, 3, 2, 1, 0, 5, 6, 8, 4]  Heuristic value ->  10
State ->  [0, 7, 2, 1, 3, 5, 6, 8, 4]  Heuristic value ->  8
State ->  [7, 2, 0, 1, 3, 5, 6, 8, 4]  Heuristic value ->  10
State ->  [0, 3, 2, 7, 1, 5, 6, 8, 4]  Heuristic value ->  8
State ->  [7, 3, 2, 6, 1, 5, 0, 8, 4]  Heuristic value ->  10
State ->  [7, 3, 2, 1, 0, 5, 6, 8, 4]  Heuristic value ->  10
State ->  [4, 1, 